# Giving controls

In [71]:
# VEL=0
# import pytesseract as pyt
def straight():
#     print('forward')
    PressKey(W)
    ReleaseKey(A)
    ReleaseKey(D)
    time.sleep(1)
#     ReleaseKey(W)

    
def left():
#     print('left')
    PressKey(A)
    ReleaseKey(W)
    ReleaseKey(D)
    time.sleep(1)
#     ReleaseKey(A)

    
def right():
#     print('right')
    PressKey(D)
    ReleaseKey(A)
    ReleaseKey(W)
    time.sleep(1)
#     ReleaseKey(D)
    
    
    
def status(o, out):
    if(o==0):
        return 'left '+str(np.round(out[0],2))+' '+str(np.round(out[1],2))+' '+str(np.round(out[2],2))
    if(o==1):
        return 'forward'+str(np.round(out[0],2))+' '+str(np.round(out[1],2))+' '+str(np.round(out[2],2))
    if(o==2):
        return 'right'+str(np.round(out[0],2))+' '+str(np.round(out[1],2))+' '+str(np.round(out[2],2))
    
    
def self_drive():

    for i in range(3):
        print(i+1)
        time.sleep(1)

    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (50, 50)
    fontScale = 1
    color = (255, 0, 0)
    thickness = 2
    
    while(True):
        screen = pyautogui.screenshot(region=(0,40,800,600))
        speed = pyt.image_to_string(screen[600:650, 400:450])
        screen = process_image(screen)
        x = np.zeros((1, 120,120,1), np.int)
        t = cv2.resize(screen,(120,120))
        x[:,:,:,0]=t
        out = model.predict(x)[0]
        o=np.argmax(out)
        screen=cv2.putText(screen, status(o,out), org, font, 
                   fontScale, color, thickness, cv2.LINE_AA)
        cv2.imshow('vision',screen)
        if(cv2.waitKey(10) & 0xFF==ord('q')):
            cv2.destroyAllWindows()
            break
        if(o==0):
            left()
        if(o==1 & int(speed)<20):
            straight()
        if(o==2):
            right()

In [72]:
self_drive()


1
2
3


TypeError: 'Image' object is not subscriptable

In [ ]:
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_gen = ImageDataGenerator(rescale=1./255)
tester = ImageDataGenerator(rescale=1./255)
train_data = train_gen.flow_from_directory('base/train',target_size=(256, 256) ,batch_size=50, class_mode='categorical')
test_data = tester.flow_from_directory('base/valid',target_size=(256, 256) ,batch_size=50, class_mode='categorical')

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), input_shape=(256, 256, 3), activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.fit(train_data, epochs=10, validation_data=test_data, steps_per_epoch=90, validation_steps=20)

In [ ]:
main()

## Collecting Data for training

In [14]:
# -*- coding: utf-8 -*-
# Citation: Box Of Hats (https://github.com/Box-Of-Hats )

import win32api as wapi
import win32con as wcon

keyList = ["\b"]
for char in "ABCDEFGHIJKLMNOPQRSTUVWXYZ 123456789,.'£$/\\":
    keyList.append(char)


def key_check():
    keys = []
    for key in keyList:
        if wapi.GetAsyncKeyState(ord(key)):
            keys.append(key)
    if wapi.GetAsyncKeyState(wcon.VK_UP):
        keys.append('up')
    if wapi.GetAsyncKeyState(wcon.VK_DOWN):
        keys.append('down')
    if wapi.GetAsyncKeyState(wcon.VK_RIGHT):
        keys.append('right')
    if wapi.GetAsyncKeyState(wcon.VK_LEFT):
        keys.append('left')
    if wapi.GetAsyncKeyState(wcon.VK_SPACE):
        keys.append('space')

    return keys



# def keys_to_output(keys):
#     """
#     Convert keys to a ...multi-hot... array
#     ['up - 0', 'down - 1', 'left - 2', 'right - 3', 'none - 4']
#     """
#     output = [0, 0, 0, 0, 0]

#     if 'left' in keys:
#         output[2] = 1
#     elif 'up' in keys:
#         output[0] = 1
#     elif 'down' in keys:
#         output[1] = 1
#     elif 'right' in keys:
#         output[3] = 1
#     else:
#         output[4] = 1

#     return output

# create_training_data.py



import os


def keys_to_output(keys):
    '''
    Convert keys to a ...multi-hot... array
    [A,W,D] boolean values.
    '''
    output = [0,0,0]
    
    if 'A' in keys:
        output[0] = 1
    elif 'D' in keys:
        output[2] = 1
    else:
        output[1] = 1
    return output


file_name = 'training_data_Long.npy'

if os.path.isfile(file_name):
    print('File exists, loading previous data!')
    training_data = list(np.load(file_name, allow_pickle=True))
else:
    print('File does not exist, starting fresh!')
    training_data = []

import PIL

def prepare_data(training_data):
    
    print('starting in: ')
    for i in list(range(5))[::-1]:
        print(i+1)
        time.sleep(1)

    
    paused = False
    time.sleep(5)
    while(True):
        if not paused:
            screen =  pyautogui.screenshot(region=(0,50,800,650))
            screen = process_image(screen)
            cv2.imshow('rec', screen)
            cv2.waitKey(30)
            screen = cv2.resize(screen, (160,120))
            keys = key_check()
            output = keys_to_output(keys)
            training_data.append([np.array(screen),np.array(output)])
            
            if len(training_data) % 500 == 0:
                print(len(training_data))
                np.save(file_name,training_data)

        keys = key_check()
        if 'T' in keys or 't' in keys:
            
            cv2.destroyAllWindows()
            
            if paused:
                paused = False
                print('unpaused!')
                time.sleep(1)
            else:
                print('Pausing!')
                paused = True
                time.sleep(1)
                

File does not exist, starting fresh!


In [15]:
prepare_data(training_data)

starting in: 
5
4
3
2
1
500
Pausing!
unpaused!
Pausing!
unpaused!
1000
Pausing!


KeyboardInterrupt: 

### Data cleaning (removing extra W inputs)

In [ ]:
def seperate(IMAGES, KEYS):
    n = len(KEYS)
    left=[]
    forw=[]
    right=[]
    for i in range(n):
        if(KEYS[i][0]==1):
            left.append([IMAGES[i], KEYS[i]])
        elif(KEYS[i][1]==1):
            forw.append([IMAGES[i], KEYS[i]])
        elif(KEYS[i][2]==1):
            right.append([IMAGES[i], KEYS[i]])
    return left, forw, right
            

In [ ]:
data = np.load('training_data.npy', allow_pickle=True)

IMAGES=[]
KEYS=[]

for d in data:
    IMAGES.append(d[0])
    KEYS.append(d[1])

In [ ]:
l,f,r = seperate(IMAGES, KEYS)

In [ ]:
print('left', len(l))
print('forw', len(f))
print('right', len(r))

In [ ]:
def multiple(keys):
    aw=0
    wd=0
    for key in keys:
        if(key[0]==1 & key[1]==1):
            aw=aw+1
        elif(key[1]==1 & key[2]==1):
            wd=wd+1
    return aw, wd


print(multiple(KEYS))

In [ ]:
import random

n=max(len(l),len(r))
forward=[]
for i in range(n):
    forward.append(random.choice(f))

l_tr = l[:-500]
forward_tr = forward[:-500]
r_tr = r[:-500]

l_ts = l[-500:]
forward_ts = forward[-500:]
r_ts = r[-500:]

In [ ]:
import shutil 
import os

try:
    shutil.rmtree('base')
    shutil.rmtree('train')
    shutil.rmtree('test')
    
except:
    pass


base='base/'
os.mkdir('base/')
train_path = 'base/train/'
os.mkdir('base/train')
os.mkdir('base/train/0')
os.mkdir('base/train/1')
os.mkdir('base/train/2')
test_path = 'base/test/'
os.mkdir('base/test')
os.mkdir('base/test/0')
os.mkdir('base/test/1')
os.mkdir('base/test/2')


In [ ]:
from PIL import Image as im

for i in range(len(l_tr)):
    data = im.fromarray(l_tr[i][0])
    data.save(train_path+'0/'+str(i)+'.jpg')
    
for i in range(len(forward_tr)):
    data = im.fromarray(forward_tr[i][0])
    data.save(train_path+'1/'+str(i)+'.jpg')
    
for i in range(len(r_tr)):
    data = im.fromarray(r_tr[i][0])
    data.save(train_path+'2/'+str(i)+'.jpg')
    
    
for i in range(len(l_ts)):
    data = im.fromarray(l_ts[i][0])
    data.save(test_path+'0/'+str(i)+'.jpg')
    
for i in range(len(forward_ts)):
    data = im.fromarray(forward_ts[i][0])
    data.save(test_path+'1/'+str(i)+'.jpg')
    
for i in range(len(r_ts)):
    data = im.fromarray(r_ts[i][0])
    data.save(test_path+'1/'+str(i)+'.jpg')
  
 

## Creating model 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_gen = ImageDataGenerator(rescale=1./255)
tester = ImageDataGenerator(rescale=1./255)
train_data = train_gen.flow_from_directory('train',target_size=(256, 256) ,batch_size=50, class_mode='categorical')
test_data = tester.flow_from_directory('valid',target_size=(256, 256) ,batch_size=50, class_mode='categorical')

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), input_shape=(256, 256, 3), activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.fit(train_data, epochs=10, validation_data=test_data, steps_per_epoch=90, validation_steps=20)

In [55]:
# import time
# import numpy as np
# import cv2
# import tensorflow as tf
# import pyautogui
data = np.load('training_data_Long.npy', allow_pickle=True)

images=[]
keys =[]
w=120
h=120
for d in data:
    img = cv2.resize(d[0], (w,h))
    images.append(img)
    keys.append(d[1])
    
    
images = np.array(images)
keys = np.array(keys)

In [118]:
import pytesseract as pyt

In [ ]:
main()

In [11]:
import numpy as np
import cv2
import time
import pyautogui
import ctypes

SendInput = ctypes.windll.user32.SendInput


W = 0x11
A = 0x1E
S = 0x1F
D = 0x20

# C struct redefinitions 
PUL = ctypes.POINTER(ctypes.c_ulong)
class KeyBdInput(ctypes.Structure):
    _fields_ = [("wVk", ctypes.c_ushort),
                ("wScan", ctypes.c_ushort),
                ("dwFlags", ctypes.c_ulong),
                ("time", ctypes.c_ulong),
                ("dwExtraInfo", PUL)]

class HardwareInput(ctypes.Structure):
    _fields_ = [("uMsg", ctypes.c_ulong),
                ("wParamL", ctypes.c_short),
                ("wParamH", ctypes.c_ushort)]

class MouseInput(ctypes.Structure):
    _fields_ = [("dx", ctypes.c_long),
                ("dy", ctypes.c_long),
                ("mouseData", ctypes.c_ulong),
                ("dwFlags", ctypes.c_ulong),
                ("time",ctypes.c_ulong),
                ("dwExtraInfo", PUL)]

class Input_I(ctypes.Union):
    _fields_ = [("ki", KeyBdInput),
                 ("mi", MouseInput),
                 ("hi", HardwareInput)]

class Input(ctypes.Structure):
    _fields_ = [("type", ctypes.c_ulong),
                ("ii", Input_I)]

# Actuals Functions



def PressKey(hexKeyCode):
    extra = ctypes.c_ulong(0)
    ii_ = Input_I()
    ii_.ki = KeyBdInput( 0, hexKeyCode, 0x0008, 0, ctypes.pointer(extra) )
    x = Input( ctypes.c_ulong(1), ii_ )
    ctypes.windll.user32.SendInput(1, ctypes.pointer(x), ctypes.sizeof(x))

def ReleaseKey(hexKeyCode):
    extra = ctypes.c_ulong(0)
    ii_ = Input_I()
    ii_.ki = KeyBdInput( 0, hexKeyCode, 0x0008 | 0x0002, 0, ctypes.pointer(extra) )
    x = Input( ctypes.c_ulong(1), ii_ )
    ctypes.windll.user32.SendInput(1, ctypes.pointer(x), ctypes.sizeof(x))

if __name__ == '__main__':
    PressKey(0x11)
    time.sleep(1)
    ReleaseKey(0x11)
    time.sleep(1)

def roi(image, vertices):
    mask = np.zeros_like(image)
    mask = cv2.rectangle(mask, (0,200), (800, 600), 255, -1)
#     mask = cv2.rectangle(mask, (0, 450),(180, 600), 0, -1)
    masked = cv2.bitwise_and(image, mask)
    return masked



def process_image(image):
    img = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)
    img = cv2.Canny(img, threshold1=100, threshold2=250)
    
    
    vertices = [[0,0],[0,200],[800,200],[800,0]]
    vertices = np.array(vertices)
    img = roi(img, vertices)
    
    
    lines = cv2.HoughLinesP(img, 1, np.pi/180, 150,  None, 10, 0)
    img = cv2.GaussianBlur(img, (5,5), 0)
    x = np.zeros_like(img)
    lines = clean_lines(lines)
    
    if(lines!=None): 
#         draw_lines(x, lines)
        draw_lines(img, lines)
    return img

def XY(Y_new,m,c):
    X_new = (Y_new-c)//m
    if(X_new<0):
        X_new=0
        Y_new=c
    if(X_new>800):
        X_new=800
        Y_new=m*X_new+c
        
    return round(X_new), round(Y_new)


def clean_lines(lines):
    try:
        good_lines=[]
        for line in lines:
            line=line[0]
            x,y,X,Y = line
            slope_limit = 0.1
            m=abs(slope(x,y,X,Y))
            if(m<=slope_limit or m>2):
                pass
            else:
                good_lines.append([line])
        return good_lines
    except: 
        return None


def slope(x,y,X,Y):
    num = Y-y
    deno = X-x
    if(deno==0): return 0
    return num/deno


def chk(x,y,X,Y):
    if(y==200):
        if(X<200 or X>600):
            return True
    if(x==0):
        if(y<350):
            return True
    return False
            


def draw_lines(img, lines):
    for line in lines:
        x,y,X,Y = line[0][0],line[0][1],line[0][2],line[0][3]
        m=slope(line[0][0],line[0][1],line[0][2],line[0][3])
        c=y-m*x
        X_new, Y_new = XY(200,m,c)
        x_new, y_new = XY(600,m,c)        
        if(not chk(x_new,y_new,X_new,Y_new)): cv2.line(img, (x_new,y_new), (X_new,Y_new), [255,255,255], 1)


def main():
    last_time = time.time()
    i=0
    while True:
        img = pyautogui.screenshot(region=(0,50,800,600))
        present = time.time()
        while(i<10):
            print("fps: ",1/(present-last_time))
            i=i+1
        crop = img.crop((670,465,700,500))
        crop = cv2.resize(np.array(crop), (300,350))
        cv2.imshow('crop', np.array(crop))
          frame = np.array(img)
#         frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        edges = process_image(frame)
#         cv2.imshow("screenshot", a)
        cv2.imshow('edges', edges)
        rec=edges
        if (cv2.waitKey(30) == ord("q")):
            cv2.destroyAllWindows()
            break

            

In [17]:
# images = np.expand_dims(images, axis=-1)
# keys = np.expand_dims(keys, axis=-1)

In [56]:
print(images.shape)

(1000, 120, 120)


In [58]:
x = np.zeros((1000, 120,120,1), np.int)
x[:,:,:,0]=images

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), input_shape=(120, 120, 1), activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

model.fit(x, keys, epochs=15, batch_size=16)